In [6]:
# This code requires sentences.txt to contain a set of scientific implications of the form P implies Q.
# Copy environment.txt or medical.txt to sentences.txt as needed
# It then tries all 8 combinations of P, Q and its negations and tests how times the LLM says the answer is correct.


# Because of limits to number of queries per day. I use the gemini-2-5-flash lite model as with the free account you get 1000 free queries a day for my region/account.
# The number of free queries you get may differ and if it is less I suggest not having 100 queries in sentences.txt
# If you start getting "too many calls" errors, then it's because you've exceeded your limit.


# Core libraries
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from scipy.linalg import eigh

# ===========================
# Load and preprocess dataset
# ===========================

from google.colab import drive
drive.mount('/content/drive/')



import requests
import json
import time
import os
from google.colab import userdata

# The API key will be provided by the environment.
# We use os.environ.get as a good practice, but it's not strictly necessary for Canvas.
API_KEY = userdata.get('GOOGLE_API_KEY')
API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent"


def check_implication(sentence,PQ,posP,posQ):
    """
    Checks if a sentence of the form "P implies Q" is true using an LLM.

    Args:
        sentence (str): The input sentence containing "implies".

    Returns:
        bool: True if the LLM's response indicates the implication is true, False otherwise.
              Returns None if the API call fails or the sentence is malformed.
    """
    # Split the sentence into P and Q based on the word "implies"
    parts = sentence.split(" implies ")
    if len(parts) != 2:
        print(f"Skipping malformed sentence: '{sentence}'. Expected 'P implies Q' format.")
        return None

    P = parts[0].strip()
    Q = parts[1].strip()

    # Construct the LLM query
    # Put "No" before P and Q and interchange P and Q to construct all 8 combinations
    if not posP:
        P = f"No {P}"
    if not posQ:
        Q = f"No {Q}"

    if PQ:
        query = f"Is it true that '{P}' implies '{Q}'? Answer with 'yes' or 'no'."
    else:
        query = f"Negating: Is it true that '{Q}' implies '{P}'? Answer with 'yes' or 'no'."

#    print(f"Checking: '{query}'")

    # Define the payload for the API request
    payload = {
        "contents": [
            {
                "parts": [
                    {
                        "text": query
                    }
                ]
            }
        ],
        "tools": [
            {"google_search": {}}
        ]
    }

    headers = {
        "Content-Type": "application/json"
    }

    retries = 5 # Increased retries
    delay = 2 # Increased initial delay
    for i in range(retries):
        try:
            # Make the API call
            response = requests.post(f"{API_URL}?key={API_KEY}", headers=headers, data=json.dumps(payload))
            response.raise_for_status()  # Raise an exception for bad status codes

            result = response.json()
            candidate = result.get("candidates", [])[0]
            response_text = candidate.get("content", {}).get("parts", [])[0].get("text", "").lower()
#            print(f"Response: {response_text}")

            # Check the response for "yes" or "true"
            if "yes" in response_text or "true" in response_text:
                return True
            return False

        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            if i < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2  # Exponential backoff
            else:
                print("Max retries exceeded. Aborting.")
                return None
        except (IndexError, KeyError) as e:
            print(f"Failed to parse API response: {e}")
            print(f"Response content: {response.text}")
            return None

def main(file_path):
    """
    Reads sentences from a file, checks each implication, and prints the count.

    Args:
        file_path (str): The path to the file containing the sentences.
    """
    sentences = []
    try:
        with open(file_path, 'r') as file:
            sentences = [line.strip() for line in file if line.strip()]
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return

    for PQ in [False, True]:
        for posP in [False, True]:
            for posQ in [False, True]:
                yes_count = 0
                print("Checking implications for P=>Q with setting for PQ, negP and negQ\n", {PQ}, {posP}, {posQ})
                for i, sentence in enumerate(sentences):
                    is_true = check_implication(sentence,PQ,posP,posQ)

                    if is_true is not None:
                        if is_true:
                            yes_count += 1
#                            print("Result: Yes\n")
#                        else:
#                            print("Result: No\n")

                    # Add a delay between API calls to avoid hitting rate limits
                    if is_true is not None: # Added delay even if API call fails
                        time.sleep(2)  # Increased delay

                print("-" * 30)
                print(f"Total implications checked: {len(sentences)}")
                print(f"Number of 'yes' answers: {yes_count}")

if __name__ == "__main__":
    main('/content/drive/My Drive/sentences.txt')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

for m in genai.list_models():
  print(f"Name: {m.name}")
  print(f"Description: {m.description}")
  print(f"Supported Generation Methods: {m.supported_generation_methods}")
  print("-" * 20)